In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# 0. 사용할 패키지 불러오기
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.layers import Activation
from keras.layers import Dropout

# 랜덤시드 고정시키기
np.random.seed(8)

In [29]:
# 1. 데이터 생성하기
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/drive/My Drive/Colab Notebooks/crop/train',
        target_size=(105,500),
        batch_size=8,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        '/content/drive/My Drive/Colab Notebooks/crop/test',
        target_size=(105,500),    
        batch_size=8,
        class_mode='categorical')

Found 520 images belonging to 9 classes.
Found 52 images belonging to 9 classes.


In [36]:
y_train=np.array([0,1,2,3,4,5,6,7,8])
y_train

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [0]:
# 2. 모델 구성하기
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(105,500, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 103, 498, 32)      896       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 101, 496, 32)      9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 50, 248, 32)       0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 50, 248, 32)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 48, 246, 64)       18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 46, 244, 64)       36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 122, 64)      

In [0]:
# 3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
# 4. 모델 학습시키기
model.fit_generator(
        train_generator,
        steps_per_epoch=60,
        epochs=75,
        validation_data=test_generator,
        validation_steps=15)

Epoch 1/75
60/60 [==============================] - 4s 68ms/step - loss: 3.2309 - acc: 0.2125 - val_loss: 1.5805 - val_acc: 0.4911
Epoch 2/75
60/60 [==============================] - 3s 46ms/step - loss: 1.3693 - acc: 0.4792 - val_loss: 1.0157 - val_acc: 0.6250
Epoch 3/75
60/60 [==============================] - 3s 44ms/step - loss: 0.9360 - acc: 0.6375 - val_loss: 0.7431 - val_acc: 0.7411
Epoch 4/75
60/60 [==============================] - 3s 46ms/step - loss: 0.7204 - acc: 0.7500 - val_loss: 0.7665 - val_acc: 0.6964
Epoch 5/75
60/60 [==============================] - 3s 46ms/step - loss: 0.5760 - acc: 0.7938 - val_loss: 0.4551 - val_acc: 0.8393
Epoch 6/75
60/60 [==============================] - 3s 46ms/step - loss: 0.4307 - acc: 0.8250 - val_loss: 0.3491 - val_acc: 0.8839
Epoch 7/75
60/60 [==============================] - 3s 47ms/step - loss: 0.4883 - acc: 0.8187 - val_loss: 0.7337 - val_acc: 0.7685
Epoch 8/75
60/60 [==============================] - 3s 45ms/step - loss: 0.5366 - a

In [0]:
# 5. 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=20)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))


-- Evaluate --
acc: 96.05%


In [0]:
# 6. 모델 사용하기
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)


-- Predict --
{'a4': 0, 'b4': 1, 'c4': 2, 'c5': 3, 'd4': 4, 'e4': 5, 'f4': 6, 'g4': 7, 'zero': 8}
[[0.000 0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 1.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 1.000 0.000 0.000]
 [0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 0.000 1.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 1.000 0.000 0.000]
 [0.000 0.000 1.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 1.000 0.000 0.000]
 [1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [1.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000]
 [0.000 0.000 0.000 0.000 0.000 0.000 1.000 0.0

In [0]:
from keras.models import load_model

model.save('virtual_piano_model_final_2.h5')

In [0]:
piano_model_json = model.to_json()
with open("piano_model.json", "w") as json_file : 
    json_file.write(piano_model_json)

In [0]:
model.save_weights("virtual_piano_model_weight.h5")
print("Saved model to disk")

Saved model to disk
